Library

In [ ]:
import os
import glob
import cv2

import file

In [ ]:
def pathProcessing (path):
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif']  # Add more extensions if needed
    image_paths = []

    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(path, ext)))

    return image_paths

In [ ]:
def readImage (paths):
    images=[]

    for path in paths:
        image= cv2.imread(path)
        if image is not None:
            images.append(image)
        else:
            print(f"Failed to read image at path: {path}")

    return images

PreProcessing

In [ ]:
def gaussianBlur(images):
    for image in images:
    print(image)

K mean Cluster

Tracing

Result